![Ironhack logo](https://i.imgur.com/1QgrNNw.png)


# Supervised Learning Project

**by Octavio Garcia**

A continuación se detallan las fases desarrolladas dentro de un proyecto de 'aprendizaje supervisado multiclase'. Los requerimientos del proyecto son los siguientes:

- Problema multiclase (utilización del módulo OneVsRestClassifier de sklearn).

- Análisis previo de los datos (balance entre clases).

- Entrenamiento de 2 modelos de ml.

- Extracción de métricas: Accuracy, Precision, Recall, F1Score, ROC, AUC, Confussion Matrix.

Para responder a los requerimientos se ha realizado un 'pipeline' que incluye:

- **Fase 1:** Elección y análisis inicial del dataset.

- **Fase 2:** Preparación del dataset para evaluación de los modelos elegidos.

- **Fase 3:** Obtención de métricas.



In [ ]:
#Importanción de librerías utilizadas en el desarrollo del proyecto.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from scipy import interp

## Fase 1: Elección y análisis inicial del dataset.

Se ha elegido un dataset que incluye las estadísticas de baseball, concretamente las estadísticas de bateo por jugador de toda la historia de la Major League Baseball. Dichas estadísticas son valores numéricos que corresponden al desempeño ofensivo de cada jugador durante una temporada completa, sumando un total de 17 estadísticas de bateo más otros dos valores numéricos (año y stint, el cual es un identificador de si el jugador ha estado en más de un equipo en un año).

Asimismo, el dataset incluye features categóricas que corresponden al identificador del jugador, el equipo en el que juega y la liga a la que pertenece el equipo. El clasificador se aplicará sobre el equipo donde juega el jugador de manera que el modelo sea capaz de predecir en que equipo ha jugado un jugador en función de sus estadísticas de bateo. 

![Ironhack logo](moneyball.jpg)

In [ ]:
#Importanción del dataset.
bat = pd.read_csv('./Batting.csv')#,encoding = "ISO-8859-1")

In [ ]:
#Visualización del dataset.
bat.head()

In [ ]:
#Tamaño del dataset (105861 registros con 22 columnas)
bat.shape

In [ ]:
#Tipo de datos del dataset
bat.dtypes

In [ ]:
#Visualización de valores nulos.
bat.isna().sum()

In [ ]:
#Análisis de las variables categóricas. Se crea una función para analizar las variables categóricas.
def selected_feat(selected,df):
    values = []
    leng = selected.shape
    for i in range(leng[0]):
        values.append(df[selected[i]].unique())
        new_df = pd.DataFrame(values).T
    new_df.columns = [i for i in selected]
    return new_df

In [ ]:
#Variables categóricas a análizar.
object_col = np.array(['teamID','lgID'])

In [ ]:
#Resultado del análisis de las variables categóricas.
selected_feat(object_col,bat).head(10)

In [ ]:
#Eliminación de los registros con valores nulos.
bat.dropna(inplace=True)

In [ ]:
#Verificación de que no existen registros con valores nulos.
bat.isna().sum()

In [ ]:
#Se analizan nuevamente las columnas categóricas.
selected_feat(object_col,bat).head(10)

In [ ]:
#Se verifica cuan balancedos están los datos por equipos.
bat.teamID.value_counts()

### FILTRADO DE DATOS:

El dataset incluye estadísticas de toda la historia de la MLB, sin embargo, existen muchos equipos en los albores de la historia de la MLB que fueron desapareciendo y equipos que han cambiado de ciudad y propietario pero que realmente son los mismos (i.e.: cambio de franquicia). También, han habido cambios importantes en las reglas del juego por lo que he decidido trabajar solamente con los registros posteriores a 1975, año en el que se realizaron las últimas modificaciones importantes al juego. Este filtro también elimina muchos equipos con bajo número de registros.

In [ ]:
#Eliminación de registros anteriores a 1975.
bat_1975 = bat[bat.yearID >= 1975]

In [ ]:
#Reasignación de nombres de las columnas juntando las franquicias bajo un mismo nombre de equipo.
TEAMS = {'TEX':'Texas Rangers','NYA':'New York Yankees','CLE':'Cleveland Indians','NYN':'New York Mets',
         'SDN':'San Diego Padres','OAK':'Oakland Athletics','BOS':'Boston Red Sox','PIT':'Pittsburgh Pirates',
         'LAN':'Los Angeles Dodgers','KCA':'Kansas City Royals','BAL':'Baltimore Orioles',
         'PHI':'Philadelphia Phillies','CIN':'Cincinnati Reds','ATL':'Atlanta Braves','CHN':'Chicago Cubs',
         'SFN':'San Francisco Giants','SEA':'Seattle Mariners','DET':'Detroit Tigers','CHA':'Chicago White Sox',
         'SLN':'St. Louis Cardinals','TOR':'Toronto Blue Jays','HOU':'Houston Astros','MIN':'Minnesota Twins',
         'MON':'Montreal Expos','COL':'Colorado Rockies','ARI':'Arizona Diamondbacks','TBA':'Tampa Bay Rays',
         'MIL':'Milwaukee Brewers','CAL':'Los Angeles Angels','ML4':'Milwaukee Brewers','FLO':'Miami Marlins',
         'WAS':'Washington Nationals','LAA':'Los Angeles Angels','MIA':'Miami Marlins','ANA':'Los Angeles Angels'}
bat_1975.replace({'teamID':TEAMS},inplace=True)
#Ahora el total de equipos es el total de equipos actuales más los Expos de Montreal que desaparecieron en 2004.
bat_1975['teamID'].unique()

In [ ]:
bat_1975.teamID.value_counts()

In [ ]:
#Drop pitchers or no-batters
bat_AB = bat_1975[bat_1975.AB != 0]#.reset_index()

In [ ]:
champs = pd.read_csv('./Champions.csv')

In [ ]:
ws_df = pd.DataFrame(champs.CHAMPION.value_counts().reset_index())
ws_df.rename(columns={'index':'Team','CHAMPION':'WorldSeriesTotal'},inplace=True)
ws_df

In [ ]:
ws_df['Team'].str.contains(bat_1975['teamID'].unique()[0]).any()

In [ ]:
def WorldSeries(team):
    equipo = str(team)
    if ws_df['Team'].str.contains(equipo).any() == True:
        if (ws_df.loc[ws_df['Team'] == equipo, 'WorldSeriesTotal'].iloc[0]) > 3:
            team = 'G1 World Series Team'
        elif 2 <= (ws_df.loc[ws_df['Team'] == equipo, 'WorldSeriesTotal'].iloc[0]) <= 3:
            team = 'G2 World Series Team'
        elif (ws_df.loc[ws_df['Team'] == equipo, 'WorldSeriesTotal'].iloc[0]) == 1:
            team = 'G3 World Series Team'
        return team
    else:
        team = "No World Series Team"
        return team

In [ ]:
WorldSeries('Cincinnati Reds')

In [ ]:
bat_AB['WorldSeries'] = bat_AB['teamID'].apply(WorldSeries)

In [ ]:
bat_AB.head()

In [ ]:
bat_AB.WorldSeries.value_counts()

In [ ]:
#Dropping useless columns
bat_df = bat_AB.drop(['playerID','lgID','teamID'],axis=1)

In [ ]:
STATS = {'G':'Games','AB':'At Bat','R':'Runs Scored','H':'Hits','2B':'Double','3B':'Triple',
         'HR':'Home Runs','RBI':'Run Batted In','SB':'Stolen Base','CS':'Caught Stealing',
         'BB':'Base on Balls','SO':'Strike Out','IBB':'Intentional Base on Balls','HBP':'Hit By Pitch',
         'SH':'Sacrifice Hit','SF':'Sacrifice Fly','GIDP':'Double Plays Induced'}
bat_df.rename(columns=STATS,inplace=True)

In [ ]:
bat_df.head()

In [ ]:
bat_df.shape

In [ ]:
bat_df.dtypes

In [ ]:
X_columns = [col for col in bat_df.columns.values if col != 'WorldSeries']
X = bat_df[X_columns]
display(X.shape,X.head(),X.columns)

In [ ]:
y = pd.get_dummies(data=bat_df['WorldSeries'])
display(y.shape,y.head())

In [ ]:
'''
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.set()
ax = sns.heatmap(X.corr())
'''
mask = np.zeros_like(X.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(12,10))
ax = sns.heatmap(X.corr(), mask=mask, annot=True, cmap="BuPu")

In [ ]:
X.drop(['Games', 'At Bat', 'Runs Scored', 'Hits', 'Double',
       'Home Runs', 'Run Batted In', 'Caught Stealing',
       'Base on Balls', 'Strike Out', 
       'Sacrifice Fly',
       'Double Plays Induced'],axis=1,inplace=True)

In [ ]:
'''
a4_dims = (11.7, 8.27)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.set()
ax = sns.heatmap(X.corr())
'''

mask = np.zeros_like(X.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(12,10))
ax = sns.heatmap(X.corr(), mask=mask, annot=True, cmap="BuPu")

In [ ]:
#Logistic Regression multilabel model function with crossvalidation
def LRs(X,y,ns):
    cls = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
    scores = cross_val_score(cls,X,y,cv=ns)
    return print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
LRs(X,y,5)

In [ ]:
#Support Vector Machines multilabel model function with crossvalidation
def SVMs(X,y,ns):
    cls = OneVsRestClassifier(svm.SVC(kernel='rbf',probability=True,gamma='auto'))
    scores = cross_val_score(cls,X,y,cv=ns)
    return print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
SVMs(X,y,5)

In [ ]:
#Random Forest multilabel model function with crossvalidation
def RFs(X,y,n,ns):
    cls = OneVsRestClassifier(RandomForestClassifier(n_estimators=n))
    scores = cross_val_score(cls,X,y,cv=ns)
    return print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
RFs(X,y,20,5)

In [ ]:
#K Neighbors multilabel model function with crossvalidation
def KNNs(X,y,k,ns):
    cls = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=k))
    scores = cross_val_score(cls,X,y,cv=ns)
    return print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
KNNs(X,y,10,5)

## Model 1 (Random Forest)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=42)
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
cls = OneVsRestClassifier(RandomForestClassifier(n_estimators=20))
cls.fit(X_train,y_train)

In [ ]:
y_pred = cls.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

In [ ]:
predictions = cls.predict(X_test).argmax(axis=1)
actual_values = y_test.values.argmax(axis=1)
print(predictions,actual_values)

In [ ]:
cm = confusion_matrix(actual_values, predictions)
cm

In [ ]:
precision = pd.DataFrame(precision_score(actual_values, predictions, average=None))
recall = pd.DataFrame(recall_score(actual_values, predictions, average=None))
f1_score = pd.DataFrame(f1_score(actual_values, predictions, average=None))
display(precision,recall,f1_score)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize=(12,10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
plot_confusion_matrix(actual_values, predictions, y.columns, cmap='BuPu')

In [ ]:
plot_confusion_matrix(actual_values, predictions, y.columns, normalize=True, cmap='BuPu')